# TPCH Hybrid Query 3 Example

In this TPCH Hybrid Query 3 Example our goal is:

&emsp; Explore how Wayang supports the execution of queries over multiple data stores systematically
- Wayang works with data produced by different sources
- Wayang is able manipulate and integrate data from different formats

---

## Preparing dependancies


<!--  Importing the needs libraries for the exections, this are comming from the maven instalation perfomed in the step 0 -->

<!-- > This step imports the necessary libraries for subsequent program executions.
All these packages come from the previous Maven Instalation -->

This step imports the required modules to execute the code. All these packages come from the previous Maven Instalation

The imported libraries are:

Module | Java's | Scala's | Description
:----- | -------------: | --------------: | :----------
wayang-core | 8, 11 | 2.11, 2.12 | provides core data structures and the optimizer (required)
wayang-basic | 8, 11 | 2.11, 2.12 | provides common operators and data types for your apps (recommended)
wayang-api-scala-java | 8, 11 | 2.11, 2.12 | provides an easy-to-use Scala and Java API to assemble wayang plans (recommended)
wayang-java | 8, 11 | 2.11, 2.12 | adapters for [Java Stream](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.html) processing platforms
wayang-spark | 8, 11 | 2.11, 2.12 | adapters for [Apache Spark](https://spark.apache.org) processing platforms
wayang-flink | 8, 11 | 2.11, 2.12 | adapters for [Apache Flink](https://flink.apache.org) processing platforms
hadoop-common | 8,11 | - | Hadoop-commons is required because the lack of the Environment Variable **HADOOP_HOME**


In [1]:
/* Import Dependencies */
import $ivy.`com.thoughtworks.paranamer:paranamer:2.8`

import $ivy.`org.apache.wayang::wayang-api-scala-java:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-core:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-basic:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-java:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang::wayang-spark:0.6.1-SNAPSHOT`
import $ivy.`org.apache.hadoop:hadoop-common:2.8.5`
import $ivy.`org.apache.wayang::wayang-benchmark:0.6.1-SNAPSHOT`

/* Include required classes */
import org.apache.wayang.api._
import org.apache.wayang.core.api.{Configuration, WayangContext}
import org.apache.wayang.core.util.fs.FileSystems
import org.apache.wayang.java.Java
import org.apache.wayang.apps.tpch.CsvUtils
import org.apache.wayang.apps.tpch.data.{Customer, LineItem, Order}
import org.apache.wayang.apps.util.ExperimentDescriptor
import org.apache.wayang.apps.tpch.data.LineItem
import org.apache.wayang.core.plugin.Plugin
import org.apache.wayang.jdbc.operators.JdbcTableSource
import org.apache.wayang.jdbc.platform.JdbcPlatformTemplate
import org.apache.wayang.postgres.Postgres
import org.apache.wayang.postgres.operators.PostgresTableSource
import org.apache.wayang.sqlite3.Sqlite3
import org.apache.wayang.sqlite3.operators.Sqlite3TableSource
import org.apache.wayang.apps.util.Parameters
import java.io.File;

import $ivy.$                                                        

import $ivy.$                                             

import $ivy.$                                              

import $ivy.$                                             

import $ivy.$                                               

import $ivy.$                                      

import $ivy.$                                                   

/* Include required classes */

import org.apache.wayang.api._

import org.apache.wayang.core.api.{Configuration, WayangContext}

import org.apache.wayang.core.util.fs.FileSystems

import org.apache.wayang.java.Java

import org.apache.wayang.apps.tpch.CsvUtils

import org.apache.wayang.apps.tpch.data.{Customer, LineItem, Order}

import org.apache.wayang.apps.util.ExperimentDescriptor

import org.apache.wayang.apps.tpch.data.LineItem

import org.apache.wayang.core.plugin.Plugin

import org.apache.wayang.jdbc.operators.JdbcTableSource

import org.apache.wayang.jd

## Wayang Context 

Wayang Context is the entry point for users to work with Wayang. Allows to declare several execution Platforms to run the code.

- The configurated property 'wayang.postgres.jdbc.url' defines the JDBC connection to Postgres instances

- The object Java.basicPlugin enables to use Wayang Operators on Java Platform
- The object Postgres.plugin enables to use Wayang Operators on Postgres Database

> WayangContext also manages the Job creation and its execution as Wayang Plans

In [2]:
val configuration = new Configuration

/* Defines JDBC connection to Postgres */
configuration.setProperty("wayang.postgres.jdbc.url", "jdbc:postgresql://{{THE_IP_FOR_POSTGRES}}:{{THE_PORT_FOR_POSTGRES}}/{{THE_DATABASE_NAME_FOR_POSTGRES}}?user={{THE_USER_FOR_POSTGRES}}&password={{THE_PASSWORD_FOR_POSTGRES}}")

/* Adds required plugins to process different data sources */
val context = new WayangContext(configuration)
                    .withPlugin(Java.basicPlugin)
                    .withPlugin(Postgres.plugin)

configuration: Configuration = Configuration[(no name)]
context: WayangContext = org.apache.wayang.core.api.WayangContext@1cc231a5

## Hybrid TPCH 3 Query - Multi Platform Execution

- Plan Builder works as an utility to build and execute Wayang Plans 
- A Wayang Plan consists of a set of operators with dependencies between them
- The provided logical plan is independant of any underlying platform to use
> Wayang performs an optimization process over this Wayang plan, transforming this logical plan in a set of Physical operators to be executable by specific platforms

In [3]:
/* Results format */
case class Query3Result(orderKey: Long, var revenue: Double, orderDate: Int, shipPriority: Int)

/* Query to be executed */
def tpch3hybrid(context: WayangContext) = {
    
    /* Setting TPCH3 query parameters */
    val segment: String = "BUILDING"
    val _segment = segment
    val date: String = "1995-03-15"
    val _date = CsvUtils.parseDate(date)
    
    /* Tables/Tuples fetching functions */
    val createTableSource = (table: String, columns: Seq[String]) => new PostgresTableSource(table, columns: _*)
    def schemaObj = Map("CUSTOMER" -> "customer", "ORDERS" -> "orders", "LINEITEMS" -> "lineitems")
    def withSchema(table: String) = schemaObj match {
      case null => table
      case str => s"$str.$table"
    }
    
    /* Builder to prepare and execute the plan */
    val planBuilder = new PlanBuilder(context)
    
    
    /* From Postgres Table 'CUSTOMER' read, filter, and project the customer data */
    val customerKeys = planBuilder
      .readTable(createTableSource(schemaObj("CUSTOMER"), Customer.fields))
      .withName("Load CUSTOMER table")
    
      .filter(_.getString(6) == _segment, sqlUdf = s"c_mktsegment LIKE '$segment%'", selectivity = .25)
      .withName("Filter customers")

      .projectRecords(Seq("c_custkey"))
      .withName("Project customers")

      .map(_.getLong(0))
      .withName("Extract customer ID")
    
    
    /* From Postgres Table 'ORDERS' read, filter, and project the order data */
    val orders = planBuilder
      .load(createTableSource(schemaObj("ORDERS"), Order.fields))
      .withName("Load ORDERS table")

      .filter(t => CsvUtils.parseDate(t.getString(4)) > _date, sqlUdf = s"o_orderdate < date('$date')")
      .withName("Filter orders")

      .projectRecords(Seq("o_orderkey", "o_custkey", "o_orderdate", "o_shippriority"))
      .withName("Project orders")

      .map(order => (order.getLong(0), // orderKey
        order.getLong(1), // custKey
        CsvUtils.parseDate(order.getString(2)), // orderDate
        order.getInt(3)) // shipPriority
      )
      .withName("Unpack orders")

    
    /* Register Line Item input file */
    val lineitemFile = new File("files/lineitem.tbl").toURI().toString()

    /* From Text File 'lineitem.tbl' read, filter, and project the line item data. */
    val lineItems = planBuilder
      .readTextFile(lineitemFile)
      .withName("Read line items")
      .map(LineItem.parseCsv)
      .withName("Parse line items")

      .filter(_.shipDate > _date)
      .withName("Filter line items")

      .map(li => (li.orderKey, li.extendedPrice * (1 - li.discount)))
      .withName("Project line items")

    
    /* Join and aggregate the different datasets. */
    customerKeys
      .join[(Long, Long, Int, Int), Long](identity, orders, _._2)
      .withName("Join customers with orders")
      .map(_.field1) // (orderKey, custKey, orderDate, shipPriority)
      .withName("Project customer-order join product")

      .join[(Long, Double), Long](_._1, lineItems, _._1)
      .withName("Join CO with line items")
      .map(coli => Query3Result(
        orderKey = coli.field1._1,
        revenue = coli.field1._2,
        orderDate = coli.field0._3,
        shipPriority = coli.field0._4
      ))
      .withName("Project CO-line-item join product")

      .reduceByKey(
        t => (t.orderKey, t.orderDate, t.shipPriority),
        (t1, t2) => {
          t1.revenue += t2.revenue;
          t2
        }
      )
      .withName("Aggregate revenue")
      .collect()
  
}

defined class Query3Result
defined function tpch3hybrid

In [4]:
var result = tpch3hybrid(context)

result: Iterable[Query3Result] = Buffer(
  Query3Result(5825025L, 12495.581999999999, 728276, 0),
  Query3Result(1973990L, 49319.239199999996, 728231, 0),
  Query3Result(5939301L, 12511.356, 728189, 0),
  Query3Result(5329221L, 63623.0048, 728239, 0),
  Query3Result(2802630L, 79222.3525, 728188, 0),
  Query3Result(2573153L, 4321.1664, 728267, 0),
  Query3Result(1919462L, 36900.234, 728254, 0),
  Query3Result(2677248L, 35677.25, 728278, 0),
  Query3Result(3382532L, 53203.04, 728248, 0),
  Query3Result(2674023L, 18901.094399999998, 728176, 0),
  Query3Result(1269440L, 58512.8286, 728262, 0),
  Query3Result(166627L, 62115.435, 728198, 0),
  Query3Result(5318023L, 41438.087999999996, 728218, 0),
  Query3Result(4333509L, 47058.191999999995, 728221, 0),
  Query3Result(3566374L, 1101.569, 728166, 0),
  Query3Result(5012320L, 37985.6477, 728266, 0),
  Query3Result(4617957L, 49724.7296, 728233, 0),
  Query3Result(3092128L, 22051.142399999997, 728181, 0),
  Query3Result(1853479L, 75308.0832, 728193, 0),
  Query3Result(5926212L, 8605.859999999999, 728277, 0),
  Query3Result(4310470L, 27116.735399999998, 728228, 0),
  Query3Result(2321381L, 39681.536, 728241, 0),
  Query3Result(1442471L, 1403.37, 728207, 0),
  Query3Result(2210499L, 61851.21599999999, 728271, 0),
  Query3Result(5949447L, 33937.25, 728181, 0),
  Query3Result(960960L, 7428.195, 728238, 0),
  Query3Result(2426853L, 40069.293600000005, 728214, 0),
  Query3Result(1029574L, 82548.83, 728275, 0),
  Query3Result(800033L, 38192.238, 728236, 0),
  Query3Result(3685316L, 53716.77, 728182, 0),
  Query3Result(5640769L, 55044.877199999995, 728268, 0),
  Query3Result(2591782L, 72428.076, 728259, 0),
  Query3Result(5201859L, 14226.6592, 728162, 0),
  Query3Result(5314147L, 57717.8836, 728260, 0),
  Query3Result(2926531L, 27643.815, 728182, 0),
  Query3Result(1722050L, 56154.7503, 728238, 0),
  Query3Result(1319457L, 81503.2656, 728262, 0),
  Query3Result(5165157L, 47376.252, 728185, 0),
...